In [ ]:
import os
import pandas as pd
from collections import defaultdict

def load_dictionary(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file if line.strip()]

def count_words_in_files(data_directory, dictionary):
    # 初始化一个字典来保存每个文件的计数结果
    word_counts_per_file = defaultdict(lambda: defaultdict(int))
    total_words_per_file = defaultdict(int)  # 新增一个字典来保存每个文件的总词数
    # 遍历每年的数据
    for year in range(2007, 2023):
        year_folder = os.path.join(data_directory, str(year))
        if os.path.exists(year_folder):
            for file_name in os.listdir(year_folder):
                file_path = os.path.join(year_folder, file_name)
                file_word_counts = defaultdict(int, {word: 0 for word in dictionary})  # 初始化所有词汇计数为0
                with open(file_path, 'r', encoding='utf-8') as file:
                    words = file.read().split()
                    total_words_per_file[file_name] = len(words)  # 更新文件的总词数
                    for word in words:
                        if word in dictionary:
                            file_word_counts[word] += 1
                word_counts_per_file[file_name] = file_word_counts
    return word_counts_per_file, total_words_per_file

def create_excel(data_directory, dictionary, excel_path):
    # 获取每个文件的词频统计及总词数
    word_counts_per_file, total_words_per_file = count_words_in_files(data_directory, dictionary)
    
    # 创建一个列表，包含每个文件的统计数据，以及文件名中的代码和日期
    rows = []
    for file_name, word_counts in word_counts_per_file.items():
        # 解析文件名获取代码和日期
        code, date = file_name.split('_')
        row = {'Code': code, 'Date': date}
        row.update(word_counts)
        row['Total Words in File'] = total_words_per_file[file_name]  # 新增每个文件的总词数
        rows.append(row)
    
    # 初始化DataFrame
    df = pd.DataFrame(rows)
    
    # 填充缺失值为0
    df.fillna(0, inplace=True)
    
    # 对于字典中的每个词，添加一个列到DataFrame中
    for word in dictionary:
        if word not in df:
            df[word] = 0
            
    # 重新排序列，将Code和Date放在前面
    cols = ['Code', 'Date'] + dictionary + ['Total Words in File']
    df = df[cols]
    
    # 导出到Excel
    df.to_excel(excel_path, index=False)

# 字典文件路径
dictionary_file_path = '/数据/创新文化词典.txt'

# 数据文件夹路径
data_directory = '/数据/去除停用词和分词'

# Excel文件保存路径
excel_path = '/数据/创新文化.xlsx'

# 加载词典
dictionary = load_dictionary(dictionary_file_path)

# 创建并导出Excel
create_excel(data_directory, dictionary, excel_path)
